## Neural Analog Modelling 
### -- Introduction
This repo is concerned with modelling analog effects with neural networks. Why? Mostly for curiosity.

Most VST effects are typically implemented in C++, using handy frameworks like JUCE which contains libraries to handle many of the typical challenges of plugin design such as cross-platform functionality, DSP, frontend design.. In order to do real analog modelling, one typically needs to have some high level domain knowledge and have the capacity to understand complicated analog circuits present in analog synths and effects, as well as a knowledge of DSP which allows us to model these signals numerically. Other approaches include simulation of physical processes like reverberation.

In principle, neural networks are a more high level approach which, presupposing the access to a relevant dataset of dry/wet signals, allow us to model the effects without having to bust out Korg manuals from the 1980s and examine the intricacies of the circuits for their wonderful filters. 

There are clearly many limits in using neural networks to process audio. Some include:
- Clean datasets of dry/wet signals are not typically easy to come by, and are labour intensive to repair. Hence NNs are not always a suitable approach for modelling analog effects.
- Neural networks are typically quite bulky, and implemented in Python (which is typically slower compared to DSP implementations in C++). Although not impossible (see work of [C. Steinmetz](https://scholar.google.com/citations?user=jSvSfIMAAAAJ&hl=en)), this makes it difficult to use neural networks to process signals in real time. This inherent slowness makes a lot of neural approaches only suitable to asynchronous audio processing. Sequence to sequence audio modelling is notoriously slow, especially given the large size of common audio models used today e.g [Demucs](https://github.com/facebookresearch/demucs) (for source separation). 

I am interesting in exploring these limits, especially the second one. 

### -- Data 
In this case, we use the [SignalTrain](https://zenodo.org/records/3824876) dataset from 2019 which contains various dry and wet recordings, where the wet recordings are processed with an analog compressor, the Universal Audio LA-2A. 

This compressor is a very simple one, and we will be concerned with modelling the signal using only two parameters on the compressor:
- The switch between compression and limiting
- The peak reduction

The information about the parameters of the compressor are contained in the file names. In particular, the value between 0-100 represents the peak reduction knob, whereas the binary value 0/1 represents the switch between compression (0) and limiting (1). The authors say that there was no changes to the input or output gains, and that only these two parameters above were changed during recording.

### -- Audio
The audio in this dataset has a sampling rate of 44.1kHz and is mono, as the original analog LA-2A was designed to process mono signals. The individual audio files actually contain a "collage" of different pieces of music which are stitched seamlessly together. The whole recording will be passed through the compressor to obtain the wet signal.

I followed the advice of the authors in cleaning up errors in the dataset (removing / moving certain files) in other notebook `data_validation.ipynb`. I cross correlated all the signals to see if any signals except those mentioned by the authors had a phase shift between the dry and wet signals. To simplify my life, I found all the signals which had a relative phase shift and removed them, keeping only perfectly correlated dry/ wet pairs; which to the credit of the authors, was the vast majority of the signals.

## Compressors

In [ ]:
import numpy as np 

def compress(threshold, ratio):
    